In [60]:
import getpass
import os
import dotenv

dotenv.load_dotenv()

True

In [61]:
from langchain.chat_models import init_chat_model

model = init_chat_model("gpt-4o-mini", model_provider="openai")


In [62]:
from langchain.prompts import PromptTemplate

prompt = PromptTemplate.from_template(
    """
    Analyze the following code and identify security vulnerabilities.
Return the output in JSON format with the following fields:
- "line": The line number where the issue occurs.
- "token": The specific code token that is problematic.
- "problem": A short name for the issue.
- "explanation": A brief explanation of why this is a security risk.
- "migration": A brief explanation of how to fix this issue.
Code:
{code}
"""
)

In [63]:
code_snippet = '''
contract Vault {
    using ECDSA for bytes32;

        address public owner;

    constructor() payable {
        owner = msg.sender;
    }

    function transfer(address to, uint amount, bytes[2]memory sigs) external {
        bytes32 hashed = computeHash(to, amount);
        require(validate(sigs, hashed), "invalid sig");

        (bool sent, ) = to.call{ value: amount } ("");
        require(sent, "Failed to send Ether");
    }

    function computeHash(address to, uint amount) public pure returns(bytes32) {
        return keccak256(abi.encodePacked(to, amount));
    }

    function validate(bytes[2]memory sigs, bytes32 hash) private view returns(bool) {
        bytes32 ethSignedHash = hash.toEthSignedMessageHash();

        address signer = ethSignedHash.recover(sigs[0]);
        bool valid = signer == owner;

        if (!valid) {
            return false;
        }

        return true;
    }
}
'''

In [64]:
from langchain.chains import LLMChain
chain = chain = prompt | model

In [65]:
result = chain.invoke({"code": code_snippet})
result

AIMessage(content='```json\n[\n    {\n        "line": 13,\n        "token": "to.call{ value: amount } (\\"\\")",\n        "problem": "Reentrancy Attack",\n        "explanation": "The use of the low-level call to send Ether can expose the contract to reentrancy attacks if the recipient is a contract that calls back into the Vault during execution.",\n        "migration": "Use the Checks-Effects-Interactions pattern or consider using OpenZeppelin\'s ReentrancyGuard contract to protect against reentrancy attacks."\n    },\n    {\n        "line": 9,\n        "token": "require(validate(sigs, hashed), \\"invalid sig\\");",\n        "problem": "Insufficient Signature Validation",\n        "explanation": "The signature validation checks if only the owner signed the transaction but does not check if the signature is indeed valid for the specific transaction, allowing potential malicious actions if the owner\'s key is compromised.",\n        "migration": "Implement a robust multi-signature or ti

In [66]:
import json
print(result.content.strip('`json'))


[
    {
        "line": 13,
        "token": "to.call{ value: amount } (\"\")",
        "problem": "Reentrancy Attack",
        "explanation": "The use of the low-level call to send Ether can expose the contract to reentrancy attacks if the recipient is a contract that calls back into the Vault during execution.",
        "migration": "Use the Checks-Effects-Interactions pattern or consider using OpenZeppelin's ReentrancyGuard contract to protect against reentrancy attacks."
    },
    {
        "line": 9,
        "token": "require(validate(sigs, hashed), \"invalid sig\");",
        "problem": "Insufficient Signature Validation",
        "explanation": "The signature validation checks if only the owner signed the transaction but does not check if the signature is indeed valid for the specific transaction, allowing potential malicious actions if the owner's key is compromised.",
        "migration": "Implement a robust multi-signature or time-lock mechanism and validate the signature a